In [20]:
from orders.models  import Order, OrderDetail
from orders.serializers import OrderSerializer, OrderDetailSerializer
csv_arr = []
import os
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../'))+'assets/'
queryset = Order.objects.filter(id=1)
data = OrderSerializer(queryset.last())
from common.models import State

state = State.objects.get(id = 1)
email = state.depo_email.split(',')
print(email)
filename = '/code/staticfiles/order_csv/padip.csv'
print(filename)

from django.core.mail import send_mail
from django.utils.html import strip_tags
from django.core.mail import EmailMultiAlternatives
from django.template.loader import render_to_string
from django.utils.html import strip_tags

for orderlist in data.data['order_details']:
    for order in orderlist['orders']:
        arr = {}
        arr['Ecatalogue_orderno'] = data.data['orderId']
        arr['Ecatalogue_orderDate'] = data.data['addedon']
        arr['Customer_Code'] = data.data['user']['id']
        arr['Shipping_point'] = ""
        arr['line_no'] = 1
        arr['Item_Code'] = order['product']['item_code']
        arr['Item_Variant'] = order['product']['variant']
        arr['Quantity'] = order['quantity']
        arr['Remarks'] = ""
        csv_arr.append(arr)
        
def send_email(Order):
    header  =  '<html xmlns="http://www.w3.org/1999/xhtml">'
    header  +=  '<head>'
    header  +=  '<link href="//maxcdn.bootstrapcdn.com/bootstrap/3.3.0/css/bootstrap.min.css" rel="stylesheet" id="bootstrap-css">'
    header  +=  '<script src="//maxcdn.bootstrapcdn.com/bootstrap/3.3.0/js/bootstrap.min.js"></script>'
    header  += '<script src="//code.jquery.com/jquery-1.11.1.min.js"></script>'

    header  +=  '</head>'

    header  +=  '<body>'
    header  +=  '<div id="mailsub" class="notification" align="center">'
    header  +=  '<div class="col-xs-12">'
    header  +=     '<img height="100" style="text-align: center" src="http://ecat.jaispring.com/assets/images/logo.png">'
    header  += '</div>'
    header  += '<div class="col-xs-12" style="padding-top: 10px;font-size: 30px;text-align: center">'
    header  +=  '<span style="font-weight: bolder;font-weight: bolder;font-size: 24px;line-height: 81px;float: revert;" CLASS="model-head-detail">E-CAT CONNECT - SALE ORDER</span>'

    header  += '</div>'
    
    header  += '<div class="col-xs-12" style="border-top:1px solid grey;padding-top: 10px">'
    header += '<table class="table" border="1">'
    header += '<thead style="background:#0B9FDB" class="table-primary">'
    header += '<tr class="table-primary">'
    header += '<th colspan="2">SALSE ORDER NUMBER</th>'
    header += '<th colspan="2">SALSE ORDER DATE</th>'
    header += '</tr>'
    header += '</thead>'
    header += '<tbody>'
    header += '<tr>'
    header += '<td colspan="2">'
    header +=    str(Order["orderId"])+'</td><td colspan="2">'+str(Order["addedon"])+'</td></tr></tbody><thead style="background:#0B9FDB" class="table-primary"><tr class="table-primary"><th>CUSTOMER NAME </th> <th>MOBILE</th><th>ADDRESS</th></tr></thead><tbody><tr>'
    header += '<td>'+str(Order["user"]["first_name"])+ str(Order["user"]["last_name"])+'</td>'
    header +=     '<td>'+str(Order["addedon"])+'</td>'
    header +=     '<td>'+str(Order["user"]['profile']['contact_number'])+'</td>'
    header +=     '<td>'+str(Order["user"]['profile']['address1'])+'</td></tr></tbody></table>'
    for orderOne in Order['order_details']:
        header += '<table class="table" border="1">'
        header += '<thead class="table-primary" style="background:#0B9FDB;line-height: 50px">'
        header +=                 '<th style="text-align: center" colspan="10" class="table-primary">'
        header +=                   str(orderOne["name"])
        header +=                 '</th>'
        header +=             '</thead>'
        header +=             '<thead style="background:#ccc" class="table-primary">'
        header +=                 '<tr class="table-primary">'
        header +=                     '<th>ID</th>'
        header +=                     '<th>CODE</th>'
        header +=                     '<th>VARIENT</th>'
        header +=                     '<th>SEGMENT</th>'
        header +=                     '<th>SUB SEGMENT</th>'
        header +=                     '<th>DESC</th>'
        header +=                     '<th>PRICE</th>'
        header +=                     '<th>QUANTITY</th>'
        header +=                     '<th>ORDER VALUE</th>'
        header +=                 '</tr>'
        header +=           ' </thead>'
        for order in orderOne["orders"]:
            header +=             '<tbody>'
            header +=                 '<tr>'
            header +=                     '<td>'+str(order["id"])+'</td>'
            header +=                     '<td>'+str(order["product"]["item_code"])+'</td>'
            header +=                     '<td>'+str(order["product"]["variant"])+'</td>'
            header +=                     '<td>'+str(order["product"]["segment"]['name'])+'</td>'
            header +=                     '<td>'+str(order["product"]["subsegment"]['name'])+'</td>'
            header +=                     '<td>'+str(order["product"]["item_description"])+'</td>'
            header +=                     '<td>'+str(order["product"]["mrp1"])+'</td>'
            header +=                     '<td>'+str(order["quantity"])+'</td>'
            header +=                     '<td>'+str(order["product"]["mrp1"]*order["quantity"])+'</td>'
            header +=                 '</tr>'
            header +=             '</tbody>'
        header +=           ' </table>'
    
    header  +=         '<table class="table" border="1">'
    header  +=         '<thead style="background:#0B9FDB" class="table-primary">'
    header  +=         '<tr class="table-primary">'
    header  +=          '<th colspan="2"></th>'
    header  +=          '<th colspan="2"></th>'
    header  +=                 '</tr>'
    header  +=             '</thead>'
    header  +=             '<tbody>'
    header  +=                 '<tr>'
    header  +=                     '<td colspan="2">'
    header  +=                         '<strong>GRAND Total</strong>'
    header  +=                     '</td>'
    header  +=                     '<td colspan="2">'
    header  +=                        '<strong>'+str(Order["total"])+'</strong>'
    header  +=                     '</td>'
    header  +=                 '</tr>'
    header  +=             ' </tbody>'
    header  +=       '</table>'
    header  +=       '</div>'
    header  += '</div>'
    header  += '</body>'
    header  += '</html>'
    order_html = header
    subject, from_email, to = 'hello', 'pradip.comat@gmail.com', 'pradip.comat@gmail.com'
    msg = EmailMultiAlternatives(subject, html_content, from_email, [to])
    msg.attach_alternative(html_content, "text/html")
    msg.send()
def write_to_csv(input_json,file_name):
    import csv
    file_path = file_name
    with open(file_path, 'w', newline="") as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        header = [key for key, value in input_json[0].items()]
        csv_writer.writerow(header)
        for item in input_json[1:]:
            row = [v for k, v in item.items()]
            csv_writer.writerow(row)  
# send_email(data.data)
write_to_csv(csv_arr,filename)            

['jaiconnect@jaispring.com', 'pradip.comat@gmail.com']
/code/staticfiles/order_csv/padip.csv
